# CARE Score and Care2CompareDataset usage

This notebook shows how to apply the CAREScore to the CARE2Compare dataset ([zenodo](https://doi.org/10.5281/zenodo.10958774)).
Contents of this notebook:

1. Using the Care2CompareDataset class to load the CARE to compare dataset from zenodo.
2. Using the CAREScore to evaluate a model on the dataset.
3. Recreating the results of the CARE paper.
4. Using Care2CompareDataset and CARE-Score for other datasets.

In [ ]:
import os
from pathlib import Path

import pandas as pd

from energy_fault_detector.fault_detector import FaultDetector
from energy_fault_detector.config import Config
from energy_fault_detector.evaluation import CAREScore, Care2CompareDataset

In [ ]:
data_dir = Path('..') / '..' / 'Care_To_Compare_v6'

In [ ]:
c2c = Care2CompareDataset(path=data_dir, download_dataset=False)  # If you have not downloaded the dataset yet, set download_dataset to True

In [ ]:
c2c.event_info_all

In [ ]:
# select data for a specific event
x, y = c2c.get_dataset_for_event(0, statistics=['average', 'std_dev'])
x.head()

### Create model per event and evaluate

In [ ]:
c2c = Care2CompareDataset(data_dir)
index_column = 'id'  # us time_stamp as index column if you are using the TimestampTransformer
suffix = ''

configs = {
    'A': Config('c2c_configs/windfarm_A.yaml'),
    'B': Config('c2c_configs/windfarm_B.yaml'),
    'C': Config('c2c_configs/windfarm_C.yaml'),
}

result_files = {
    'A': f'results_A{suffix}.csv',
    'B': f'results_B{suffix}.csv',
    'C': f'results_C{suffix}.csv',
}

# For WF A we cannot use the normal index to filter the events, as this would remove complete anomalous events (these are not based on actual SCADA Status codes).
# For WF B and C this info is however useful, and it would not be interesting to detect non-actionable anomalies (normal_index=False)
ignore_normal_idx = {
    'A': True,
    'B': False,
    'C': False,
}

# Initialize CARE-Score
care_score = CAREScore(coverage_beta=0.5, reliability_beta=0.5, anomaly_detection_method='criticality')

for wf in ['A', 'B', 'C']:
    print('Wind Farm ', wf)
    
    # update model config
    config = configs[wf]
    c2c.update_c2c_config(config, wf)
    
    results_file = result_files[wf]
    if os.path.exists(results_file):
        care_score.load_evaluated_events(results_file)

    event_ids = c2c.event_info_all.loc[c2c.event_info_all['wind_farm'] == wf, 'event_id']
    for event_id in event_ids:

        if not care_score.evaluated_events.empty and event_id in care_score.evaluated_events['event_id'].values:
            # skip if already evaluated
            continue
        
        print("Evaluating Event", event_id)
        x_train, x_test = c2c.load_event_dataset(event_id=event_id, index_column=index_column)
        # create normal index
        y_train = x_train['status_type_id'] == 0
        y_test = x_test['status_type_id'] == 0
        
        # drop unnecessary features
        x_train = x_train.drop(['asset_id', 'status_type_id', 'time_stamp'], axis=1, errors='ignore')
        x_test = x_test.drop(['asset_id', 'status_type_id', 'time_stamp'], axis=1, errors='ignore')
        
        # create model
        model = FaultDetector(config)
        # train and predict
        train_results = model.fit(sensor_data=x_train, normal_index=y_train, save_models=False)
        prediction = model.predict(x_test)
        
        # evaluate event
        event_info = c2c.event_info_all[c2c.event_info_all['event_id'] == event_id]
        event_start = event_info['event_start_id'].iloc[0] if index_column == 'id' else event_info['event_start'].iloc[0]
        event_end = event_info['event_end_id'].iloc[0] if index_column == 'id' else event_info['event_end'].iloc[0]
        care_score.evaluate_event(
            event_id=event_id,
            event_start=event_start,
            event_end=event_end,
            event_label=event_info['event_label'].iloc[0],
            normal_index=y_test,
            predicted_anomalies=prediction.predicted_anomalies['anomaly'],
            ignore_normal_index=ignore_normal_idx[wf]
        )
        
        # save evaluated events
        care_score.save_evaluated_events(results_file)

    # print final score:
    print('Final score: ', care_score.get_final_score())

In [ ]:
# combine results and get final score over all events / wind farms
all_evaluations = pd.concat([pd.read_csv(f'results_{wf}{suffix}.csv') for wf in ['A', 'B', 'C']])
all_evaluations.to_csv(f'results_all{suffix}.csv', index=False)
care_score.load_evaluated_events(f'results_all{suffix}.csv')

print('Wind farm A:')
print(
    care_score.get_final_score(event_selection=c2c.event_info_all.loc[c2c.event_info_all['wind_farm'] == 'A', 'event_id'])
)
print('Wind farm B:')
print(
    care_score.get_final_score(event_selection=c2c.event_info_all.loc[c2c.event_info_all['wind_farm'] == 'B', 'event_id'])
)
print('Wind farm C:')
print(
    care_score.get_final_score(event_selection=c2c.event_info_all.loc[c2c.event_info_all['wind_farm'] == 'C', 'event_id'])
)

print('overall')
care_score.get_final_score()

### Create model per asset ID and evaluate each event

Another option is to create a model for asset ID, by combining the training datasets, instead of creating a model for each event separately.
We still evaluate each event separately. Example for WF A:

In [ ]:
# model config
wf = 'A'
config = Config(f'c2c_configs/windfarm_{wf}.yaml')
c2c.update_c2c_config(config, wf)
care_score = CAREScore(coverage_beta=0.5, reliability_beta=0.5, anomaly_detection_method='criticality')

for x_train, asset_id, event_ids in c2c.iter_train_datasets_per_asset(wf):
    x_train = x_train.reset_index(drop=True)

    # create normal index
    y_train = x_train['status_type_id'] == 0
    # drop unnecessary features (keep asset ID as a sort of condition)
    x_train = x_train.drop(['time_stamp', 'status_type_id'], axis=1)
    
    # create model
    model = FaultDetector(config)
    # train and predict
    train_results = model.fit(sensor_data=x_train, normal_index=y_train, save_models=False)
    
    # test and evaluate events for this asset
    for event_id in event_ids:
        print(event_id)
        # test model
        x_test = c2c.get_dataset_for_event(event_id=event_id, test_only=True)
        y_test = x_test['status_type_id'] == 0
        x_test = x_test.drop(['time_stamp', 'asset_id', 'status_type_id'], axis=1)
        prediction = model.predict(x_test)
        
        # evaluate event
        event_info = c2c.event_info_all[c2c.event_info_all['event_id'] == event_id]
        care_score.evaluate_event(
            event_id=event_id,
            event_start_id=event_info['event_start_id'].iloc[0],
            event_end_id=event_info['event_end_id'].iloc[0],
            event_label=event_info['event_label'].iloc[0],
            normal_index=y_test,
            predicted_anomalies=prediction.predicted_anomalies['anomaly'],
            ignore_normal_index=True,
        )


In [ ]:
care_score.get_final_score()

# Reproducing results from the Paper
To reproduce the results from (https://doi.org/10.3390/data9120138), an additional filter is needed (though only for wind farm C):
- determine cut-in and cut-off wind speeds by power curve analysis
- Remove potentially anomalous data from the training data:
    - Remove rows where the wind speed is outside the normal operation range (below cut-in or above cut-off)
    - Remove rows where the power is zero or near zero (e.g. $P < 0.01$).

In [ ]:
# TODO

# CARE Score and Care2CompareDataset usage on other datasets

To use the CARE-Score with other datasets you need the following data:
- define events containing anomalous data (the period before an actual fault)
- define events containing normal data
- For all events there needs to be enough training data beforehand to train an early fault detection model
- Ideally you have the same amount of normal and anomalous events

Then you can use the `CAREScore` class to calculate the final score:
- Create models and get predictions for each event.
- Evaluate each event using the `CAREScore.evaluate_event` method
- Calculate the CARE score `CAREScore.get_final_score`

For each of these events, you need to be able to train a proper model (for example one large model or a model for each event). For the CARE2Compare dataset we assumed 1 year of training data with >=70% normal operation is enough to create a normal behavior model.
